In [2]:
import os
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

In [3]:
# Set location of dataset, and locations to write preprocessed data and artifact

dataset_location = "s3://rumc-stoic-p-sagemaker-luuk-boulogne/stoic21/test/dataset/"
preprocessed_location = "s3://rumc-stoic-p-sagemaker-luuk-boulogne/stoic21/test/preprocessed/"
artifact_location = "s3://rumc-stoic-p-sagemaker-luuk-boulogne/stoic21/test/artifact/"

In [4]:
# Do preprocessing

estimator = PyTorch(  #Pytorch is not actually needed for preprocessing, 
    entry_point='do_preprocess.py',  # A python file in source_dir
    source_dir=os.getcwd(),  # local path (in SageMaker Studio)
    role=get_execution_role(),
    instance_type='ml.m5.large',
    instance_count=1,
    # volume_size=500,  # Scratch volume mounted at /tmp/ in GB
    framework_version='1.12',
    py_version='py38',
    max_run=60,  # in seconds, maximum is 5 days
    checkpoint_s3_uri=preprocessed_location,  # gets pulled from to checkpoint_local_path at the start of training, and pushed to at the end
    checkpoint_local_path= '/preprocessed/',
)

estimator.fit(
    inputs={  # Each input folder passed must have contents. 
        'dataset': dataset_location,
    }  # Paths are available on the training instance in environment variables named SM_CHANNEL_<KEY>, e.g. SM_CHANNEL_IMAGES
)

2023-02-02 12:54:24 Starting - Starting the training job...
2023-02-02 12:54:50 Starting - Preparing the instances for trainingProfilerReport-1675342464: InProgress
......
2023-02-02 12:55:50 Downloading - Downloading input data...
2023-02-02 12:56:19 Training - Downloading the training image......
2023-02-02 12:57:10 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-02-02 12:57:11,294 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-02-02 12:57:11,296 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-02-02 12:57:11,299 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-02-02 12:57:11,310 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-02-02 12:57:11,312 sagemaker_pytorch_container.t

In [5]:
# Do Training

estimator = PyTorch(
    entry_point='train.py',  # A python file in source_dir
    source_dir=os.getcwd(),  # local path (in SageMaker Studio)
    role=get_execution_role(),
    instance_type='ml.g4dn.xlarge',
    instance_count=1,
    volume_size=5,  # Scratch volume mounted at /tmp/ in GB
    framework_version='1.12',
    py_version='py38',
    max_run=600,  # in seconds, maximum is 5 days
    checkpoint_s3_uri=artifact_location,  # gets pulled from to checkpoint_local_path at the start of training, and pushed to at the end
    checkpoint_local_path= '/artifact/',  
)

estimator.fit(
    inputs={  # Each input folder passed must have contents. 
        'preprocessed': preprocessed_location,
    }  # Paths are available on the training instance in environment variables named SM_CHANNEL_<KEY>, e.g. SM_CHANNEL_IMAGES
)

2023-02-02 12:58:13 Starting - Starting the training job...
2023-02-02 12:58:39 Starting - Preparing the instances for trainingProfilerReport-1675342692: InProgress
......
2023-02-02 12:59:40 Downloading - Downloading input data...
2023-02-02 13:00:00 Training - Downloading the training image....................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-02-02 13:03:29,425 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-02-02 13:03:29,445 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-02-02 13:03:29,456 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-02-02 13:03:29,459 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-02-02 13:03:30,316 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -

UnexpectedStatusException: Error for Training job pytorch-training-2023-02-02-12-58-09-332: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "self._with_events(self._do_fit, 'fit', CancelFitException, self._end_cleanup)
 File "/opt/conda/lib/python3.8/site-packages/fastai/learner.py", line 193, in _with_events
 try: self(f'before_{event_type}');  f()
 File "/opt/conda/lib/python3.8/site-packages/fastai/learner.py", line 245, in _do_fit
 self._with_events(self._do_epoch, 'epoch', CancelEpochException)
 File "/opt/conda/lib/python3.8/site-packages/fastai/learner.py", line 240, in _do_epoch
 self._do_epoch_validate()
 File "/opt/conda/lib/python3.8/site-packages/fastai/learner.py", line 236, in _do_epoch_validate
 with torch.no_grad(): self._with_events(self.all_batches, 'validate', CancelValidException)
 File "/opt/conda/lib/python3.8/site-packages/fastai/learner.py", line 195, in _with_events
 self(f'after_{event_type}');  final()
 File "/opt/conda/lib/pyt